# SIVA(シヴァ)AI競馬開発エンジニアによる<br>「競馬で始める機械学習」ハンズオン

## 目標
    ・競馬の過去データを使って、予測AIの作成手順を身につける。
    ・前処理、アルゴリズムのチューニングによって予測精度を上げる手法を身につける。
    ・各自作成した競馬予測AIを使って本日開催の競馬の予測を行う。

## 本日の流れ
    1- 必要ライブラリの読み込み
    2- データの読み込み
    3- 前処理
    4- 予測アルゴリズムの構築
    5- 本日の競馬の予測



## 1- 必要ライブラリの読み込み

In [1]:
#データ解析ライブラリ
import pandas as pd

#数値計算ライブラリ
import numpy as np

#機械学習ライブラリ
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import util as ut
import preprocessing as pr

## 2- データの読み込み
    2016年~2018年の3年分の過去データ(data_horse_race.csv)を読み込む

In [2]:
df = pd.read_csv('data/data_horse_race.csv')

In [3]:
for c in df.columns:
    print(c,' : ',df.iloc[0][c],df[c].dtype)

開催年  :  2016 int64
開催日付  :  105 int64
競馬場コード  :  6 int64
開催回次  :  1 int64
開催日目  :  1 int64
レース番号  :  1 int64
枠番  :  1 int64
馬番  :  1 int64
血統番号  :  2013105621 int64
馬名  :  プロジェクション object
性別  :  2 int64
毛色コード  :  3 int64
馬齢  :  3 int64
東西所属コード  :  1 int64
斤量  :  540 int64
騎手コード  :  1131 int64
着順  :  16 int64
走破タイム  :  1161 int64
着差  :  29 int64
距離  :  1200 int64
芝ダート  :  d object
回り方向  :  R object
出走頭数  :  16 int64
開催年_1走前  :  2015.0 float64
開催日付_1走前  :  1114.0 float64
競馬場コード_1走前  :  3.0 float64
馬番_1走前  :  11.0 float64
斤量_1走前  :  530.0 float64
騎手コード_1走前  :  1154.0 float64
馬体重_1走前  :  474.0 float64
着順_1走前  :  14.0 float64
1コーナー順位_1走前  :  0.0 float64
2コーナー順位_1走前  :  0.0 float64
3コーナー順位_1走前  :  9.0 float64
4コーナー順位_1走前  :  10.0 float64
ラスト3Fタイム_1走前  :  400.0 float64
着差_1走前  :  29.0 float64
距離_1走前  :  1150.0 float64
芝ダート_1走前  :  d object
回り方向_1走前  :  R object
出走頭数_1走前  :  16.0 float64
天気コード_1走前  :  3.0 float64
芝馬場_1走前  :  0.0 float64
ダート馬場_1走前  :  2.0 float64


## 3- 前処理

### 目的変数の作成
    今回目的変数は[3着以下,3着以上]と設定する

In [ ]:
#着順データを抽出する
target_data = df['着順']

In [ ]:
target_data[:15]

In [ ]:
#着順がN着以内であれば1,3着以上であれば0に変換

#分類分けの閾値を設定
limit_order = 3

target_data_in_three = target_data.apply(lambda x:1 if x <= limit_order else 0)

In [ ]:
target_data_in_three[:15]

### カスタマイズ用関数
    後ほど各々作成してもらいます

In [ ]:
#カスタマイズ用前処理プログラム
def preprocessing(df):
    #make_yourself!!
    return df

In [ ]:
df_after_preprocessing = preprocessing(df)

### 不要カラムを削除

In [ ]:
def drop_columns(df):
    #不要カラムのリストを作成しまとめて削除する
    drop_columns_list = []

    #馬を特定する情報を削除
    drop_columns_list.extend(['血統番号','馬名'])

    #レース前にわからないデータを削除リストに追加
    drop_columns_list.extend(['着順','走破タイム','着差'])

    #カテゴリデータであり種類が多いため、取捨選択をよく考える必要がある
    drop_columns_list.extend(['騎手コード','騎手コード_1走前'])
    
    drop_columns_list.extend(['開催年'])


    df = df.drop(drop_columns_list,axis = 1)
    
    return df

In [ ]:
df_after_drop_columns = drop_columns(df_after_preprocessing)

### カテゴリカル変数をone-hot値に変更する
    カテゴリカルデータ : 順序性や等間隔性がないデータ
    例
    性別の以下のIDで表現した時
    牡馬 : 1 ,牝馬 : 2 , セン馬 : 3
    
    牡馬 + 牝馬 = セン馬
    
    とならない。
    

    
|馬名|性別|
| ---- | ---- |
|ディープインパクト|牡馬|
|ジェンティルドンナ|牝馬|
|カレンミロティック|セン馬|

<div align = 'center'>
    <br>
    ↓
</div>    

|馬名|性別_牡馬|性別_牝馬|性別_セン馬|
| ---- | ---- | ---- | ---- |
|ディープインパクト|1|0|0|
|ジェンティルドンナ|0|1|0|
|カレンミロティック|0|0|1|




   

In [ ]:
def to_onehot(df):
    #カテゴリカル変数のリストを作成しまとめてOne-hotベクトル化する
    dummies_columns_list = []

    dummies_columns_list.extend(['競馬場コード','性別','毛色コード','東西所属コード','芝ダート','回り方向'])

    #過去1走分データのカテゴリカル変数
    pre_col = ['競馬場コード','芝ダート','回り方向','天気コード','芝馬場','ダート馬場',]
    for c in pre_col:
        dummies_columns_list.append(c + '_1走前')

    df = pd.get_dummies(data = df, columns = dummies_columns_list)
    
    return df

In [ ]:
df_after_to_onehot = to_onehot(df_after_drop_columns)

### 空値をカラムの[0,中央値,平均値]などで埋める

In [ ]:
def fill_nan(df):
    #空値を0で埋める
    df = df.fillna(0)

    #空値を中央値で埋める
    #df = df.fillna(df.median())

    #空値を平均値で埋める
    #df = df.fillna(df.mean())
    
    return df

In [ ]:
df_after_fill_nan = fill_nan(df_after_to_onehot)

### 予測用データの作成

In [ ]:
#入力用ベクトル生成
x = df_after_fill_nan.values
#目的変数生成
y = target_data_in_three.values

### 訓練とテストにデータを分ける
    分割割合はtest_sizeで指定

In [ ]:
data_train , data_test , target_train , target_test = train_test_split(x,y,test_size=0.2)

## 4- 予測アルゴリズムの構築と精度検証

### ランダムフォレスト(分類)

In [ ]:
forest_classifier = RandomForestClassifier(random_state=0)
forest_classifier.fit(data_train, target_train)

In [ ]:
print('Train score: {}'.format(forest_classifier.score(data_train, target_train)))
print('Test score: {}'.format(forest_classifier.score(data_test, target_test)))

In [ ]:
#テストデータの内10件の予測結果
output = forest_classifier.predict(data_test)
for (t,o) in zip(target_test[:10],output[:10]):
    print('target : ',t,'predict : ',o)

### ランダムフォレスト (回帰)

In [ ]:
forest_regressor = RandomForestRegressor(random_state=0)
forest_regressor.fit(data_train, target_train)

In [ ]:
print('Train score: {}'.format(forest_regressor.score(data_train, target_train)))
print('Test score: {}'.format(forest_regressor.score(data_test, target_test)))

In [ ]:
#テストデータの内10件の予測結果
output = forest_regressor.predict(data_test)
for (t,o) in zip(target_test[:10],output[:10]):
    print('target : ',t,'predict : ',o)

## 5- 本日の競馬の予測

In [ ]:
#本日開催の競馬データの読み込み
today_data = pd.read_csv('data/today_data.csv')

In [ ]:
#先ほど作った前処理関数
df = preprocessing(today_data)
df = drop_columns(df)
df = to_onehot(df)
df = fill_nan(df)

In [ ]:
#学習時と同様のカラムを持つデータフレームを作成
df_ = pd.DataFrame([],columns = [c for c in df_after_fill_nan.columns])
df = pd.concat([df_,df])[[c for c in df_after_fill_nan.columns]]
df = df.fillna(0)

In [ ]:
df

In [ ]:
x = df.values

## 予測結果の出力

### ランダムフォレスト(分類)

In [ ]:
#予測出力
output = forest_classifier.predict(x)

In [ ]:
print(ut.course(today_data.iloc[0]['競馬場コード']),today_data.iloc[0]['レース番号'],'R')
for (t,o) in zip(today_data[['馬番','馬名']].values,output):
    print(str(t[0])+'番\t' + t[1] +'\t' +str(o))

### ランダムフォレスト(回帰)

In [ ]:
#予測出力
output = forest_regressor.predict(x)

In [ ]:
print(ut.course(today_data.iloc[0]['競馬場コード']),today_data.iloc[0]['レース番号'],'R')
for (t,o) in zip(today_data[['馬番','馬名']].values,output):
    print(str(t[0])+'番\t' + t[1] +'\t' +str(o))